# Quickstart: Semantic ranking in Azure AI Search using Python

Azure AI Search supports secondary L2 ranking that rescores initial results using machine reading comprehension models. The L2 ranker promotes more semantically relevant matches to the top.

This is the source code for the article: [Quickstart: Semantic ranking (Python)](https://learn.microsoft.com/azure/search/search-get-started-semantic).

## Install packages and set variables

In [1]:
! pip install -r requirements.txt --quiet

In [2]:
# Provide variables
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # Take environment variables from .env.

# The following variables from your .env file are used in this notebook
search_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "hotels-sample-index")

## Get the index definition

This code retrieves the hotels-sample-index and checks for an existing semantic configuration. 

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.identity import DefaultAzureCredential
import os

# Initialize the client (similar to what you already have)
search_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = "hotels-sample-index"  # or use your existing index_name variable

# Create the SearchIndexClient
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)

try:
    # Get the existing index schema
    index = index_client.get_index(index_name)
    
    print(f"Index name: {index.name}")
    print(f"Number of fields: {len(index.fields)}")
    
    # Print field details
    for field in index.fields:
        print(f"Field: {field.name}, Type: {field.type}, Searchable: {field.searchable}")
    
    # Access semantic configuration if it exists
    if index.semantic_search and index.semantic_search.configurations:
        for config in index.semantic_search.configurations:
            print(f"Semantic config: {config.name}")
            if config.prioritized_fields.title_field:
                print(f"Title field: {config.prioritized_fields.title_field.field_name}")
    else:
        print("No semantic configuration exists for this index")

except Exception as ex:
    print(f"Error retrieving index: {ex}")

Index name: hotels-sample-index
Number of fields: 14
Field: HotelId, Type: Edm.String, Searchable: False
Field: HotelName, Type: Edm.String, Searchable: True
Field: Description, Type: Edm.String, Searchable: True
Field: Description_fr, Type: Edm.String, Searchable: True
Field: Category, Type: Edm.String, Searchable: True
Field: Tags, Type: Collection(Edm.String), Searchable: True
Field: ParkingIncluded, Type: Edm.Boolean, Searchable: False
Field: LastRenovationDate, Type: Edm.DateTimeOffset, Searchable: False
Field: Rating, Type: Edm.Double, Searchable: False
Field: Address, Type: Edm.ComplexType, Searchable: None
Field: Location, Type: Edm.GeographyPoint, Searchable: False
Field: Rooms, Type: Collection(Edm.ComplexType), Searchable: None
Field: id, Type: Edm.String, Searchable: False
Field: rid, Type: Edm.String, Searchable: False
No semantic configuration exists for this index


## Add a semantic configuration to the index

This code adds a semantic configuration to an existing hotels-sample-index on your search service. No search documents are deleted by this operation and your index is still operational after the configuration is added.

In [4]:
# Add semantic configuration to hotels-sample-index and display updated index details
from azure.search.documents.indexes.models import (
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)

try:
    # Get the existing index
    existing_index = index_client.get_index(index_name)
    
    # Create a new semantic configuration
    new_semantic_config = SemanticConfiguration(
        name="semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="HotelName"),
            keywords_fields=[SemanticField(field_name="Tags")],
            content_fields=[SemanticField(field_name="Description")]
        )
    )
    
    # Add semantic configuration to the index
    if existing_index.semantic_search is None:
        existing_index.semantic_search = SemanticSearch(configurations=[new_semantic_config])
    else:
        # Check if configuration already exists
        config_exists = any(config.name == "semantic-config" 
                          for config in existing_index.semantic_search.configurations)
        if not config_exists:
            existing_index.semantic_search.configurations.append(new_semantic_config)
    
    # Update the index
    result = index_client.create_or_update_index(existing_index)
    
    # Get the updated index and display detailed information
    updated_index = index_client.get_index(index_name)
    
    print("Semantic configurations:")
    print("-" * 40)
    if updated_index.semantic_search and updated_index.semantic_search.configurations:
        for config in updated_index.semantic_search.configurations:
            print(f"  Configuration: {config.name}")
            if config.prioritized_fields.title_field:
                print(f"    Title field: {config.prioritized_fields.title_field.field_name}")
            if config.prioritized_fields.keywords_fields:
                keywords = [kf.field_name for kf in config.prioritized_fields.keywords_fields]
                print(f"    Keywords fields: {', '.join(keywords)}")
            if config.prioritized_fields.content_fields:
                content = [cf.field_name for cf in config.prioritized_fields.content_fields]
                print(f"    Content fields: {', '.join(content)}")
            print()
    else:
        print("  No semantic configurations found")
    
    print("✅ Semantic configuration successfully added!")
    
except Exception as ex:
    print(f"❌ Error adding semantic configuration: {ex}")

Semantic configurations:
----------------------------------------
  Configuration: semantic-config
    Title field: HotelName
    Keywords fields: Tags
    Content fields: Description

✅ Semantic configuration successfully added!


## Run a term query

Your first query is a keyword search, specified by `query_type` set to simple. For comparison, run this query to review the results when no semantic ranking is used. This query has search text, which provides criteria for potential matches. Output should consist of 13 documents containing literal or variant instances of the search terms.

In [7]:
from azure.search.documents import SearchClient

# Create the SearchIClient
search_client = SearchClient(endpoint=search_endpoint, credential=credential, index_name=index_name)

# Run a text query (returns a BM25-scored result set)
results =  search_client.search(query_type='simple',
    search_text="walking distance to live music" ,
    select='HotelId,HotelName,Description',
    include_total_count=True)
    
for result in results:
    print(result["@search.score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

5.5153193
2
Old Century Hotel
Description: The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.
5.074317
24
Uptown Chic Hotel
Description: Chic hotel near the city. High-rise hotel in downtown, within walking distance to theaters, art galleries, restaurants and shops. Visit Seattle Art Museum by day, and then head over to Benaroya Hall to catch the evening's concert performance.
4.8959594
4
Sublime Palace Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monu

## Run a semantic query

This example sets `query_type` to semantic. Compared to the previous query, the L2 ranker revisits the search results and rescores them using machine reading comprehension models, promoting results that are more meaningful to the query. Output should consist of 13 documents in a revised ranking. The effect of semantic ranking is apparent in the top half of the results, where results in positions four through six are more semantically aligned with the search text.

In [8]:
# Runs a semantic query (runs a BM25-ranked query and promotes the most relevant matches to the top)
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
    search_text="walking distance to live music", 
    select='HotelId,HotelName,Description', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

2.613231658935547
24
Uptown Chic Hotel
Description: Chic hotel near the city. High-rise hotel in downtown, within walking distance to theaters, art galleries, restaurants and shops. Visit Seattle Art Museum by day, and then head over to Benaroya Hall to catch the evening's concert performance.
2.271434783935547
2
Old Century Hotel
Description: The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.
1.9861756563186646
4
Sublime Palace Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches,

## Return captions

A semantic query can return captions, or a portion of the matching text. Captions are useful if content is verbose or if you want hit highlights in the results.

In shorter descriptions like the ones in this sample index, captions are less useful and you might consider omitting them if they don't add value to your solution.

In [9]:
# Runs a semantic query that returns captions
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
    search_text="walking distance to live music", 
    select='HotelId,HotelName,Description', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

2.613231658935547
24
Uptown Chic Hotel
Description: Chic hotel near the city. High-rise hotel in downtown, within walking distance to theaters, art galleries, restaurants and shops. Visit Seattle Art Museum by day, and then head over to Benaroya Hall to catch the evening's concert performance.
Caption: Chic hotel near the city. High-rise hotel in downtown, within walking distance to<em> theaters, </em>art galleries, restaurants and shops. Visit<em> Seattle Art Museum </em>by day, and then head over to<em> Benaroya Hall </em>to catch the evening's concert performance.

2.271434783935547
2
Old Century Hotel
Description: The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.
Caption: The hotel i

## Return semantic answers

A semantic query response can also includes "answers". This capability is useful if your index contains verbatim content that has the characteristics of a complete answer. To get an answer, the query must have the characteristics of a question, such as "what is" or "when is".

To meet the requirements of semantic answers, provide a query that asks a question.

Recall that answers are verbatim content pulled from your index. To get composed answers as generated by a chat completion model, considering using a [RAG pattern](https://learn.microsoft.com/azure/search/retrieval-augmented-generation-overview) or [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept).

In [10]:
# Run a semantic query that returns semantic answers  
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
 search_text="what's a good hotel for people who like to read",
 select='HotelId,HotelName,Description', query_caption='extractive', query_answer="extractive",)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: Nature is Home on the beach. Explore the shore by day, and then come home to our shared living space to relax around a stone fireplace, sip something warm, and explore the<em> library </em>by night. Save up to 30 percent. Valid Now through the end of the year. Restrictions and blackouts may apply.
Semantic Answer Score: 0.9829999804496765

2.124817371368408
1
Stay-Kay City Hotel
Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Caption: This classic hotel is<em> fully-refurbished </em>and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York on

## Clean up

If you're finished with this index, you can delete it by running the following lines. Deleting unnecessary indexes frees up space for stepping through more quickstarts and tutorials.

In [ ]:
try:
    result = index_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Confirm the index deletion by running the following script that returns the index definition. If hotels-sample-index isn't returned, you've successfully deleted the index and have completed this quickstart.

In [ ]:
try:
    result = index_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)
